<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angel Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Nicolás Becerra
- Nombre de alumno 2: Simón Sanfeliú


### **Link de repositorio de GitHub:** https://github.com/SimonSanfeliu/MDS7202-BS/tree/L7

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [3]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [4]:
def custom_features(dataframe_in):
    """
       Función que calcula las siguientes métricas para cada cliente:
       
       Length: Tiempo en días desde la primera compra hasta la última.
       Recency: Tiempo en días desde la última compra hasta el final del periodo.
       Monetary: Media del gasto por compra.
       Frequency: Número de compras.
       Periodicity: Desviación estándar del tiempo entre compras.

       Parámetros:
              dataframe_in: DataFrame de entrada.

       Salida:
              new_df: DataFrame de salida con las métricas calculadas.
    """
    #Verificando que sea un dataframe
    assert type(dataframe_in) == pd.DataFrame

    # Length
    lengths = dataframe_in.groupby("Customer ID", observed=True)["InvoiceDate"].apply(lambda x: (max(x) - min(x)).days if len(x.unique()) != 1 else 0)

    # Recency
    max_date = max(dataframe_in["InvoiceDate"]) + np.timedelta64(1, "D")  # Fecha final del dataframe + 1 día
    recencies = dataframe_in.groupby("Customer ID", observed=True)["InvoiceDate"].apply(lambda x: (max_date - max(x)).days)

    # Frecuency
    frequencies = dataframe_in.groupby("Customer ID", observed=True)["InvoiceDate"].apply(lambda x: x.nunique())

    # Monetary
    df = dataframe_in.copy()
    df["Income"] = df["Quantity"] * df["Price"]
    monetary = round(df.groupby("Customer ID", observed=True)["Income"].mean(), 2)

    # Periodicity
    # Primero se calculan los IVT para cada cliente en días
    ivts = dataframe_in.groupby("Customer ID", observed=True)["InvoiceDate"].apply(lambda x: [(x.unique()[i+1] - x.unique()[i]).days
                                                                             for i in range(len(x.unique())) if i+1 != len(x.unique())])
    periodicities = ivts.apply(lambda x: round(np.std(x), 2) if len(x) != 0 else 0)

    # Diccionario que luego será el dataframe de salida
    dic = {"Customer ID": lengths.index,
           "Length": lengths.values,
           "Recency": recencies.values,
           "Frequency": frequencies.values,
           "Monetary": monetary.values,
           "Periodicity": periodicities.values}
    
    new_df = pd.DataFrame(dic)

    return new_df

In [5]:
custom_features(df_retail)

,Customer ID,Length,Recency,Frequency,Monetary,Periodicity
0,12346.0,196,165,11,11.30,34.78
1,12347.0,37,3,2,18.64,0.00
2,12348.0,0,74,1,11.11,0.00
3,12349.0,181,43,3,26.19,72.00
4,12351.0,0,11,1,14.33,0.00
...,...,...,...,...,...,...
4309,18283.0,275,18,6,2.85,55.30
4310,18284.0,0,67,1,16.49,0.00
4311,18285.0,0,296,1,35.58,0.00
4312,18286.0,247,112,2,19.35,0.00


## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [6]:
class MinMax(BaseEstimator, TransformerMixin):
    """
        Clase que implementa la normalización MinMax.

        Parámetros:
        BaseEstimator: Clase base para todos los estimadores de sklearn.
        TransformerMixin: Clase que implementa el método fit_transform.
    """
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        minmax = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
        return minmax

### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [7]:
# Creación de la pipeline
pipeline = Pipeline([
    ("LRMFP", FunctionTransformer(custom_features)),
    ("Scaler", ColumnTransformer([("norm", MinMax(), ["Length", "Recency", "Frequency", "Monetary", "Periodicity"])])),
    ("T-SNE", TSNE())
])

# Aplicando la pipeline
new = pipeline.fit_transform(df_retail)

In [8]:
# Gráfico representación t-SNE
fig = px.scatter(
    new, x=0, y=1, title="Representación t-SNE de los datos LRMFP", height=800, width=800
)
fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [13]:
# Definiendo el estado aleatorio
RANDOM_STATE = 42

# Creando la pipeline para K-Means
def pipeline_k(k):
    """
        Función que crea la pipeline para el método de K-Means.

        Parámetros:
        k: Número de clusters.

        Salida:
        pipeline: Pipeline para el método de K-Means.
    """
    pipeline = Pipeline([
        ("LRMFP", FunctionTransformer(custom_features)),
        ("Scaler", ColumnTransformer([("norm", MinMax(), ["Length", "Recency", "Frequency", "Monetary", "Periodicity"])])),
        ("K-Means", KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init="auto"))
    ])
    return pipeline

# Obteniendo las inertias para el método del codo
inertias = [
    [k, pipeline_k(k).fit(df_retail)["K-Means"].inertia_]
    for k in range(1,21)
]
inertias = pd.DataFrame(inertias, columns=["n_clusters", "inertia"])
inertias.head(10)

,n_clusters,inertia
0,1,938.288112
1,2,389.385064
2,3,224.947166
3,4,165.390867
4,5,137.692773
5,6,115.817387
6,7,101.941918
7,8,94.514004
8,9,81.492757
9,10,72.911527


In [14]:
# Gráfico para el método del codo
px.line(
    inertias,
    x="n_clusters",
    y="inertia",
    title="Método del Codo con K-Means",
    height=600,
    markers=True,
)

> Dada la inercia de cada número de clusters, el método del codo indica que la curva empieza a suavizarse entre los valores 5 y 6. De hecho, si se observan las diferencias de inercias entre los valores vecinos, ya en 6 se aprecia esa tendencia a una paulatina disminución. Por lo tanto, se escogería un valor de k=5, con el fin de no tener un número de clusters demasiado grande y poder segmentar mejor los grupos de clientes.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

**Respuesta:**

In [15]:
# Usando k = 6
new_k = pipeline_k(6).fit(df_retail)

# Agrupando
df_new = custom_features(df_retail)
df_new["Cluster"] = new_k["K-Means"].labels_
final_df = round(df_new.groupby("Cluster").mean(numeric_only=True), 1)
final_df[""] = df_new.groupby("Cluster").count()["Length"]
final_df

,Length,Recency,Frequency,Monetary,Periodicity,
Cluster,,,,,,
0,17.1,43.9,1.7,32.0,1.7,1159
1,280.0,36.8,4.3,28.7,83.7,351
2,179.7,64.6,4.4,28.2,21.5,870
3,322.4,22.3,11.8,36.1,25.0,871
4,5.6,306.6,1.3,72.6,0.7,456
5,33.0,188.6,1.8,38.4,3.0,607


> Sí se pueden observar agrupaciones coherentes, ya que es posible generar un análisis de estos clusters y de que sus datos son muy similares entre sí. A continuación, se tiene dicho análisis para entender mejor todo esto:

> De los clusters generados, es posible ver que el más poblado (el 0) corresponde a clientes que visitaron pocas veces la tienda, pero siendo estas visitas cercanas entre sí (yendo frecuentemente), hace relativamente poco tiempo atrás y gastaron alrededor de 32 libras esterlinas. Por su length, se ve que visitaron la tienda en un periodo acotado de tiempo. De esto, se pueden tomar como clientes relativamente nuevos, que se están recién afianzando a la tienda o bien clientes esporádicos. Por su parte, el siguiente cluster más poblado (el 3), los clientes visitaron muchas veces la tienda, gastaron más dinero, conocen la tienda desde hace tiempo, visitaron hace poco y no van tan periódicamente. El siguiente cluster, el cual tiene un tamaño similar al anterior (el 2), también corresponde a clientes que visitan desde hace tiempo a la tienda, pero no han ido hace un tiempo, visitan menos, gastan menos, pero tienen una regularidad similar. Ambos clusters tienen clientes antiguos de la tienda, pero que visitan esta con distintas frecuencias y periodicidades. Sin embargo, los clientes del cluster 3 se pueden considerar como clientes impulsivos, ya que visitan hace mucho, visitan frecuentemente y su última visita fue muy reciente, teniendo que además gastan más que la mayoría de las personas (más que el cluster 0). 

> Los siguientes clusters son los más pequeños. El más grande de estos, con alrededor de 600 personas (el 5), representa a clientes que conocen hace un tiempo la tienda, pero la visitaron hace mucho tiempo atrás, de forma esporádica y pocas veces, aunque gastaron alrededor de 38 libras esterlinas. Por su parte, el cluster 4 representa a aquellos clientes que hicieron pocas visitas en un tiempo muy acotado hace mucho tiempo atrás, teniendo una baja regularidad, pero que gastaron mucho dinero. Esto podría caracterizarse como clientes mercenarios, ya que son clientes con bajo nivel de compromiso pero que aprovecharon las ofertas ofrecidas por la tienda. Finalmente, el cluster más pequeño (el 1), con alrededor de 350 personas, corresponde a aquellos que conocen la tienda hace mucho, visitaron hace poco, con una frecuencia alta, que no gastan mucho, pero son regulares. Se podría caracterizar a estos clientes como indecisos, ya que, mientras conocen la tienda hace mucho, van seguido y hace poco, sus visitas son con muchos días de diferencia entre sí y gastan poco.

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [16]:
# Gráfico representación t-SNE
fig = px.scatter(
    new, x=0, y=1, title="Representación t-SNE de los datos LRMFP", color=new_k["K-Means"].labels_, height=800, width=800
)
fig.show()

> Se puede decir que sí, ya que es posible ver claramente la tendencia de centralización de cada uno de los clusters. Esto permite ver de forma separable cada uno, teniendo fronteras en las que se mezclan un poco, pero es posible apreciar de forma clara dichas fronteras.

> Se considera que el número de clusters fue optimo también porque si se aumentara el número de estos la diferenciación se podría haber complejizado aún más, lo que haría menos separable la visualización y el problema. Por otro lado, con un numero mejor, podrían haberse combinado grupos que tienen tendencias entre sí, mermando el análisis hecho en la parte anterior.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>